In [1]:
library(data.table)
library(stringr)
library(tidyverse)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.0     v dplyr   1.0.6
v tidyr   1.1.3     v forcats 0.5.1
v readr   1.4.0     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::between()   masks data.table::between()
x dplyr::filter()    masks stats::filter()
x dplyr::first()     masks data.table::first()
x dplyr::lag()       masks stats::lag()
x dplyr::last()      masks data.table::last()
x purrr::transpose() masks data.table::transpose()



In [2]:
N = 100 # number of rows in claims
n = 10 # number of unique ids

It wouldn't be too annoying to recreate one every time, but that would involve fiddling with converting a sampling of `birth_date`s into characters, only for them to be converted back into dates right after, which seemed a bit silly.

So for that reason, and consistency as well, I created the initial dataset (not too interesting in R so I left it out), and used Excel to force the dates into the character format we want.

In [3]:
Claims=fread("claims.csv",header=T)
head(Claims)

id,birth_date,NDC
<int>,<chr>,<int64>
3,12-31-2021,6020429411
6,12-30-2021,1505506190
4,12-30-2021,2051918453
4,12-30-2021,6009829428
9,12-31-2021,7668618696
3,12-30-2021,2274652695


# (a)

The idea is to use `grepl` to get the columns that end with `_date`, and use those as our `.SD`, and then apply `parse_datetime` from tidyverse to each column.

I'll make this into a function as we'll want to use it later.

In [4]:
# change ones that end with _date to character
datify = function(Claims) {
    cols = grepl( "_date$" , names( Claims )
    Claims[,(cols) :=lapply(.SD,function(v){
      parse_datetime(v, "%m-%d-%Y")
    }), .SDcols=cols]
}

Claims = datify(Claims)

head(Claims)

id,birth_date,NDC
<int>,<dttm>,<int64>
3,2021-12-31,6020429411
6,2021-12-30,1505506190
4,2021-12-30,2051918453
4,2021-12-30,6009829428
9,2021-12-31,7668618696
3,2021-12-30,2274652695


# (b)

We can just use `str_pad` from `stringr`.

In [5]:
Claims = Claims[, NDC:=str_pad(as.character(NDC), 10, pad = "0")]
head(Claims)

id,birth_date,NDC
<int>,<dttm>,<chr>
3,2021-12-31,6020429411
6,2021-12-30,1505506190
4,2021-12-30,2051918453
4,2021-12-30,6009829428
9,2021-12-31,7668618696
3,2021-12-30,2274652695


# (c)

Have to insert a service date, doing it with a non-replacing sample insures we'll not have duplicates for a given `id`+`birth_date` pairing.

In [6]:
Claims[, service_date:=sample(seq(as.Date('2018/01/01'), as.Date('2019/12/31'), by="day"), size=.N), by=.(id, birth_date)]

Pick out which birth date is most common with custom `Mode` function applied after grouping by `id`.

In [7]:
Mode = function(x) {
    
  ux = unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

Claims[, birth_date:=lapply(.SD,Mode), by=id, .SDcols="birth_date"]


In [8]:
Claims[id==2]

id,birth_date,NDC,service_date
<int>,<dttm>,<chr>,<date>
2,2021-12-30,7210560441,2019-07-29
2,2021-12-30,4296207943,2019-06-02
2,2021-12-30,6536948038,2019-02-22
2,2021-12-30,2251484109,2019-12-10
2,2021-12-30,0033129199,2018-07-24
2,2021-12-30,3712781839,2018-04-06
2,2021-12-30,1031046859,2019-12-28
2,2021-12-30,6901920283,2018-05-29
2,2021-12-30,3025213308,2018-10-02


So as you can see, the birth date for an individual is consistent across all entries in the id now.

# (d)

We also have this consistency problem for the enrollment dataset, we want each id to have the same gender each time. The idea here will to make a preliminary `genders` dataset, which will be a table of each id and their gender. This way, we can just sample directly from this table, and the gender will still be tied to each id.

In [9]:
genders = data.table(id=1:n, gender=sample(c(1,2), size=n, replace=TRUE))
genders

id,gender
<int>,<dbl>
1,2
2,1
3,2
4,1
5,1
6,1
7,2
8,1
9,2


In [10]:
enrollment_size = 200
enrollment = genders[sample(1:n, enrollment_size, replace=TRUE), ]

And notice that doing it this way will ensure that each id has uniform gender.

In [11]:
head(enrollment[id==1])

id,gender
<int>,<dbl>
1,2
1,2
1,2
1,2
1,2
1,2


Adding the enrollment date is as simple as sampling from a range. I'll pick the range given in part e), of possible months to be enrolled in.

In [12]:
enrollment[, enroll_date:=sample(seq(as.Date('2018/07/01'), as.Date('2019/04/01'), by="day"), size=.N)]

Here's where there trick will come in: we're going to have to perform a join on id's, where we paste the earliest enrollment date to each one. So we're going to need to aggregate this enrollment data.table to get the earliest date for each id. We could use an application of the `min` function, but we can do something maybe even simpler: reorder the dates within each id grouping.

Then with chaining, we can just scrape the top row of each grouping to get the one that should be kept.

In [13]:
earliest_enrollment = enrollment[order(rank(id), enroll_date)][,.SD[c(1)],by=id]

In [14]:
Claims[enrollment[,.SD[c(1)],by=id], c("enroll_date", "gender") := list(i.enroll_date, i.gender), on = c(id="id")]

In [15]:
Claims[id==5]

id,birth_date,NDC,service_date,enroll_date,gender
<int>,<dttm>,<chr>,<date>,<date>,<dbl>
5,2021-12-31,3523030431,2019-10-24,2018-12-07,1
5,2021-12-31,3811249137,2019-12-31,2018-12-07,1
5,2021-12-31,3843957567,2019-08-18,2018-12-07,1
5,2021-12-31,1431354838,2018-11-26,2018-12-07,1
5,2021-12-31,0993917425,2019-07-05,2018-12-07,1
5,2021-12-31,3622251228,2018-05-21,2018-12-07,1
5,2021-12-31,7196237199,2018-10-31,2018-12-07,1


In [16]:
get_simple_date = function(enroll_date) {
    my_date = cut(enroll_date, "month")
    my_date
}

enrollment[, simple_date:=get_simple_date(enroll_date), by=seq_len(nrow(enrollment))]

In [17]:
head(enrollment)

id,gender,enroll_date,simple_date
<int>,<dbl>,<date>,<fct>
10,2,2018-07-12,2018-07-01
9,2,2018-09-12,2018-09-01
3,2,2018-09-09,2018-09-01
10,2,2018-08-02,2018-08-01
5,1,2018-12-07,2018-12-01
3,2,2018-07-26,2018-07-01


# (e)

I'm unsure if this data is external data provided to us, or if this is derived from the data we currently have on hand. Since it seemed relatively easy to create a dataset in this format though, I decided to create this `Monthly_Enrollment` imagining that each time someone enrolled, it counted for that month alone (even if they enrolled say on the very last day of June).

This is just an exercise in using `dcast` then to convert the long format of `enrollment` we created into a wide format. We don't want a column for every single *date* possible, just one for every month, so we have to "cut" each enrollment date short. 

In [18]:
enrollment[, simple_date:=cut(enroll_date, "month"), by=seq_len(nrow(enrollment))]
head(enrollment)

id,gender,enroll_date,simple_date
<int>,<dbl>,<date>,<fct>
10,2,2018-07-12,2018-07-01
9,2,2018-09-12,2018-09-01
3,2,2018-09-09,2018-09-01
10,2,2018-08-02,2018-08-01
5,1,2018-12-07,2018-12-01
3,2,2018-07-26,2018-07-01


This essentially tells us "yes, id 1 had an enrollment for June of 2018", and so on. Then it really is as simple as using `dcast` with the right inputs.

In [19]:
Monthly_Enrollment = dcast(enrollment,id ~ simple_date,fun.aggregate = function(x){1*any(x > 0)}, value.var = c("gender"))
head(Monthly_Enrollment)

id,2018-07-01,2018-09-01,2018-08-01,2018-12-01,2019-01-01,2018-10-01,2019-02-01,2019-03-01,2018-11-01,2019-04-01
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,1,1,1,1,0,0
2,1,1,0,1,1,1,1,1,1,0
3,1,1,1,1,1,1,1,1,1,0
4,0,1,1,1,0,1,1,1,0,0
5,1,1,0,1,1,1,1,1,1,0
6,1,1,1,1,1,1,1,0,1,0


Because we're not guaranteed for the columns to be in order (even if we sort `Monthly_Enrollment`, the first id could have not every month present), so we have to re-order the columns manually.

In [20]:
setcolorder(Monthly_Enrollment, c("id", sort(colnames(Monthly_Enrollment)[-1])))
head(Monthly_Enrollment)

id,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,0,1,1,1,1,0
2,1,0,1,1,1,1,1,1,1,0
3,1,1,1,1,1,1,1,1,1,0
4,0,1,1,1,0,1,0,1,1,0
5,1,0,1,1,1,1,1,1,1,0
6,1,1,1,1,1,1,1,1,0,0


Now here's one trick to get the `enr_length`: it's just the row sum of `Monthly_Enrollment`! To get this, perform an application to every row of the above data.table, with the `.SDcols` as every column past the first id one.

In [21]:
Monthly_Enrollment[, enr_length:=sum(as.numeric(.SD),na.rm=TRUE), .SDcols=c(2:ncol(Monthly_Enrollment)),by=1:nrow(Monthly_Enrollment)]
head(Monthly_Enrollment)

id,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,enr_length
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,0,1,1,1,1,0,8
2,1,0,1,1,1,1,1,1,1,0,8
3,1,1,1,1,1,1,1,1,1,0,9
4,0,1,1,1,0,1,0,1,1,0,6
5,1,0,1,1,1,1,1,1,1,0,8
6,1,1,1,1,1,1,1,1,0,0,8


Now imagining that we have `Monthly_Enrollment`, converting to long format is now just a `melt` task. We don't want entries that don't have an enrollment, so filter out ones with a "value" of 0.

In [22]:
Long_Monthly_Enrollment = melt(Monthly_Enrollment, id=c("id", "enr_length"), variable.name="enr_month")[value != 0]
Long_Monthly_Enrollment[id==1]

id,enr_length,enr_month,value
<int>,<dbl>,<fct>,<dbl>
1,8,2018-07-01,1
1,8,2018-08-01,1
1,8,2018-09-01,1
1,8,2018-10-01,1
1,8,2018-12-01,1
1,8,2019-01-01,1
1,8,2019-02-01,1
1,8,2019-03-01,1


Of course, now this value column is meaningless, so we can throw it away. While we're at it, we should break up `enr_month` into just year and month.

In [23]:
Long_Monthly_Enrollment[ ,`:=`(month = month(enr_month), year = year(enr_month), enr_month = NULL, value=NULL)]
Long_Monthly_Enrollment[id==1]

id,enr_length,month,year
<int>,<dbl>,<int>,<int>
1,8,7,2018
1,8,8,2018
1,8,9,2018
1,8,10,2018
1,8,12,2018
1,8,1,2019
1,8,2,2019
1,8,3,2019


# (f)

Of course these codes will have some sort of standard format, but the best I can do is randomly generate alphanumeric strings of random length, and slap them onto the claims dataset.

In [24]:
library(stringi)
code_cols = c(sprintf("code%01d", seq(1,4)))

Claims[, (code_cols):=list(stri_rand_strings(.N, sample(5:10, 5, replace=TRUE)),
                           stri_rand_strings(.N, sample(5:10, 5, replace=TRUE)),
                           stri_rand_strings(.N, sample(5:10, 5, replace=TRUE)),
                           stri_rand_strings(.N, sample(5:10, 5, replace=TRUE))) ]

In [25]:
head(Claims)

id,birth_date,NDC,service_date,enroll_date,gender,code1,code2,code3,code4
<int>,<dttm>,<chr>,<date>,<date>,<dbl>,<chr>,<chr>,<chr>,<chr>
3,2021-12-30,6020429411,2019-11-22,2018-09-09,2,LhwkAAnmX,rmDcf,MywJxCqukL,lnFky
6,2021-12-30,1505506190,2018-01-20,2018-10-13,1,jxNOvW,scBon,8yfvFm82Lv,rlBbh
4,2021-12-30,2051918453,2019-04-25,2018-12-18,1,pUSGgkjz84,cJwzkz,u2qkMnu,NdOr4089
4,2021-12-30,6009829428,2019-04-26,2018-12-18,1,VZhvRKpaS,TOfvrF,NwJYeP,V0sWGQQM
9,2021-12-30,7668618696,2019-05-16,2018-09-12,2,XhpnEAqRU,6tgiv3Cp8U,iYLMycio,kx2FA
3,2021-12-30,2274652695,2018-12-09,2018-09-09,2,nwyrxm72M,Gqesp,8xTldTx9tm,SzKNJ


## (i)

This is a simple application of the `toupper` function to the `claims` dataset. We can even reuse the `code_cols` vector to input our `.SDcols`.

In [26]:
Claims[, (code_cols) := lapply(.SD, toupper), .SDcols=code_cols ]

## (ii)

It became clear to me at this point that things would be a little weird if I didn't use a dataset that I didn't perform some manual changes to in order to make some of these checks go through. For that reason, I'm importing a dataset here rather than using the one from the previous part. Don't worry though, you've seen me generate the fake data already, and I promise I only made small changes to just the codes in order to have some of the coming checks pass. Imagine I'm just picking up where I left off, and we somehow just got lucky with the codes being ones that pass.

This is also why I needed the `datify` function, to immediately apply to the incoming raw dataset, which has character variables for dates.

In [27]:
Claims=datify(fread("full_claims.csv",header=T))

In [28]:
head(Claims)

id,birth_date,NDC,service_date,enroll_date,gender,code1,code2,code3,code4
<int>,<dttm>,<int64>,<dttm>,<dttm>,<int>,<chr>,<chr>,<chr>,<chr>
3,2021-12-30,6020429411,2018-08-26,2018-07-22,2,XTLYE1234,RJV280,WWUZ0,DUEPTZZTXU
6,2021-12-30,1505506190,2019-03-18,2019-04-25,1,O4NJIAIIVM,0209SKF,MXHBK0GVH,YXHRUMMKTC
4,2021-12-30,2051918453,2019-02-16,2018-09-16,2,GXNQHLS,K9TSDH,7D8FZ9B8MW,6DMJGEI
4,2021-12-30,6009829428,2019-03-18,2018-09-16,2,MZXZYPX,BCIY7YK2SK,GR5EBYO,8UQMYL4
9,2021-12-30,7668618696,2018-02-11,2018-07-28,2,2FIUI6SLL,BWP5WIK,DPNQIDCL,PWROT
3,2021-12-30,2274652695,2018-02-23,2018-07-22,2,SR5NY,YPT1234UHP,BXJAD,NTTPSY6XOG


In [29]:
Claims[code1 %like% "^A0"]

id,birth_date,NDC,service_date,enroll_date,gender,code1,code2,code3,code4
<int>,<dttm>,<int64>,<dttm>,<dttm>,<int>,<chr>,<chr>,<chr>,<chr>
8,2021-12-30,8063068092,2019-03-30,2019-05-31,1,A0VCZFU,QNXQ0KW66Z,SEFMD3O,UUTLU6J


## (iii)

Here's a trick: we can first concatenate all the code columns separated by a dummy character like `_`, and then just look for either "1234" or "4321" in that full string. This will tell us if we have a substring existing in *any* of the codes.

In order to check if either of two substrings exists, we can use `sapply` to check for one in a vector `str`, and then the other.

In [30]:
str = c("1234", "4321")

Claims[, flag:=any(sapply(str, grepl, do.call(paste, c(.SD, sep = "_")))), by=seq_len(nrow(Claims)), .SDcols=code_cols]

Then we just want to know if *any* flag has happened for a single id in a particular month, which just involves taking the max of all the flags in that id+year+month combo.

In [31]:
# aggregate by id, year, month level
Claims[, year:=year(service_date)]
Claims[, month:=month(service_date)]
monthly_flag = Claims[, .(flag=max(flag)) , by=.(id, year, month)]

Ordering by year+month is crucial for the flag2+group calculation step because it relies on a cumulative sum, which will of course be incorrect if the rows aren't in order within a particular id.

In [32]:
monthly_flag = monthly_flag[order(rank(id), year, month)]

Then the `flag2` is just whether the running sum has "gotten of the ground" yet, i.e. is not 0, and `group` really is just the running sum.

In [33]:
flag_calc = function(flags) {
    running_sum = cumsum(flags)
    flag2 = 1*(running_sum > 0)
    group = running_sum
    return(list(flag2, group))
}

monthly_flag[, c("flag2", "group"):=flag_calc(flag), by=id]

In [34]:
monthly_flag[order(rank(id), year, month)][id==3]

id,year,month,flag,flag2,group
<int>,<int>,<int>,<int>,<dbl>,<int>
3,2018,1,0,0,0
3,2018,2,1,1,1
3,2018,3,0,1,1
3,2018,4,0,1,1
3,2018,5,0,1,1
3,2018,6,0,1,1
3,2018,7,0,1,1
3,2018,8,1,1,2
3,2019,1,0,1,2


# (i)

In [35]:
Claims[, modifier1:=sample(c("U", "T", ""), size=N, replace=TRUE)]

Once I generated the fake `modifier1` data, I changed just one row to make the check for `Flag3` actually detect something.

In [36]:
Claims = fread("pre_flag3.csv", header=T)

In particular, I made exactly one row follow the desired `code1` format.

In [37]:
Claims[code1 %like% "^A0[A-Z0-9]*9$"]

id,birth_date,NDC,service_date,enroll_date,gender,code1,code2,code3,code4,flag,year,month,modifier1
<int>,<chr>,<int64>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<chr>
9,12/30/2021,7668618696,2/11/2018,7/28/2018,2,A02FIUI6SLL9,BWP5WIK,DPNQIDCL,PWROT,FALSE,2018,2,U


And notice it also has a `U` modifier, so this should be the one and only row with `Flag3==1`.

In [38]:
Claims[, Flag3:=(grepl(paste0("^A0[A-Z0-9]*9$"),code1) & modifier1 == "U")]

In [39]:
head(Claims)

id,birth_date,NDC,service_date,enroll_date,gender,code1,code2,code3,code4,flag,year,month,modifier1,Flag3
<int>,<chr>,<int64>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<chr>,<lgl>
3,12/30/2021,6020429411,8/26/2018,7/22/2018,2,XTLYE1234,RJV280,WWUZ0,DUEPTZZTXU,TRUE,2018,8,T,FALSE
6,12/30/2021,1505506190,3/18/2019,4/25/2019,1,O4NJIAIIVM,0209SKF,MXHBK0GVH,YXHRUMMKTC,FALSE,2019,3,,FALSE
4,12/30/2021,2051918453,2/16/2019,9/16/2018,2,GXNQHLS,K9TSDH,7D8FZ9B8MW,6DMJGEI,FALSE,2019,2,T,FALSE
4,12/30/2021,6009829428,3/18/2019,9/16/2018,2,MZXZYPX,BCIY7YK2SK,GR5EBYO,8UQMYL4,FALSE,2019,3,,FALSE
9,12/30/2021,7668618696,2/11/2018,7/28/2018,2,A02FIUI6SLL9,BWP5WIK,DPNQIDCL,PWROT,FALSE,2018,2,U,TRUE
3,12/30/2021,2274652695,2/23/2018,7/22/2018,2,SR5NY,YPT1234UHP,BXJAD,NTTPSY6XOG,TRUE,2018,2,,FALSE


In [40]:
Claims[Flag3==TRUE]

id,birth_date,NDC,service_date,enroll_date,gender,code1,code2,code3,code4,flag,year,month,modifier1,Flag3
<int>,<chr>,<int64>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<chr>,<lgl>
9,12/30/2021,7668618696,2/11/2018,7/28/2018,2,A02FIUI6SLL9,BWP5WIK,DPNQIDCL,PWROT,FALSE,2018,2,U,TRUE
